<a href="https://colab.research.google.com/github/dbtjr1103/Basic_practice/blob/main/AlexNet%EA%B5%AC%EC%A1%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models

import matplotlib.pyplot as plt
import numpy as np

In [19]:
(train_images, train_labels), (test_images, test_labels) = datasets.fashion_mnist.load_data()
train_images.shape, train_labels.shape, test_images.shape, test_labels.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [20]:
# 라벨 설정
label_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [21]:
# 채널 값을 부여 : 28, 28 -> 28, 28, 1
train_images = train_images.reshape(-1, 28, 28, 1)
test_images = test_images.reshape(-1, 28, 28, 1)
train_images.shape, test_images.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [22]:
# AlexNet 모델 입력층에 맞게 채널 3장으로 맞춤
train_images = tf.repeat(train_images, 3, axis=3)
test_images = tf.repeat(test_images, 3, axis=3)
train_images.shape, test_images.shape

(TensorShape([60000, 28, 28, 3]), TensorShape([10000, 28, 28, 3]))

In [8]:
model = models.Sequential()

# 227 x 227 x 3으로 resize
model.add(layers.experimental.preprocessing.Resizing(227, 227, 
                                                     input_shape=train_images.shape[1:]))

model.add(layers.Conv2D(96, (11, 11), strides=4, activation='relu')) # imagesize에 맞게 큰 filter에 큰 stride 적용
model.add(layers.Lambda(tf.nn.local_response_normalization)) # ReLU 결과값 안정화
model.add(layers.MaxPooling2D((3, 3), strides=2)) # Overlapping Pooling

model.add(layers.Conv2D(256, (5, 5), strides=1, activation='relu', padding='same'))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.MaxPooling2D((3, 3), strides=2))

model.add(layers.Conv2D(384, (3, 3), strides=1, activation='relu', padding='same'))

model.add(layers.Conv2D(384, (3, 3), strides=1, activation='relu', padding='same'))

model.add(layers.Conv2D(256, (3, 3), strides=1, activation='relu', padding='same'))
model.add(layers.MaxPooling2D((3, 3), strides=2))

model.add(layers.Flatten())
# model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))

# model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
# model.add(layers.Dense(1000, activation='softmax'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_2 (Resizing)       (None, 227, 227, 3)       0         
                                                                 
 conv2d_10 (Conv2D)          (None, 55, 55, 96)        34944     
                                                                 
 lambda_4 (Lambda)           (None, 55, 55, 96)        0         
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 27, 27, 96)       0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 27, 27, 256)       614656    
                                                                 
 lambda_5 (Lambda)           (None, 27, 27, 256)       0         
                                                      

In [9]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.fit(train_images, train_labels, batch_size=128, epochs=10, validation_split=0.2)

Epoch 1/10
375/375 [==============================] - 172s 434ms/step - loss: 0.8128 - accuracy: 0.6955 - val_loss: 0.3774 - val_accuracy: 0.8621
Epoch 2/10
375/375 [==============================] - 160s 426ms/step - loss: 0.4214 - accuracy: 0.8491 - val_loss: 0.3252 - val_accuracy: 0.8793
Epoch 3/10
375/375 [==============================] - 159s 425ms/step - loss: 0.3631 - accuracy: 0.8710 - val_loss: 0.3162 - val_accuracy: 0.8843
Epoch 4/10
 11/375 [..............................] - ETA: 2:22 - loss: 0.3245 - accuracy: 0.8857

KeyboardInterrupt: ignored

In [11]:
model.evaluate(test_images, test_labels)

313/313 [==============================] - 10s 31ms/step - loss: 0.3230 - accuracy: 0.8841


[0.32299500703811646, 0.8841000199317932]

In [ ]:
# AlexNet 모델을 간소화해봅시다 
## resize없이 (28, 28, 3)으로 받아봅시다

## ** activation 은 모두 relu로 유지
## layer 1 : 32개, (3, 3), strides=1 / pooling은 (2, 2) overlapping x / normalizing 유지

## layer 2 : 64개, (3, 3), strides=1 / pooling은 (2, 2) overlapping x / normalizing 유지

## layer 3 : 128개, (3, 3), strides=1 / pooling은 (2, 2) overlapping x

## layer 4, 5 : 삭제

## fully connected layer 1, 2 : node 원하는대로 주기


In [25]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), strides=1, activation='relu', input_shape=(28, 28, 3))) # imagesize에 맞게 큰 filter에 큰 stride 적용
model.add(layers.Lambda(tf.nn.local_response_normalization)) # ReLU 결과값 안정화
model.add(layers.MaxPooling2D((2, 2), strides=1)) # Overlapping Pooling

model.add(layers.Conv2D(64, (3, 3), strides=1, activation='relu', padding='same'))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.MaxPooling2D((2, 2), strides=1))

model.add(layers.Conv2D(128, (3, 3), strides=1, activation='relu', padding='same'))

# model.add(layers.Conv2D(384, (3, 3), strides=1, activation='relu', padding='same'))

# model.add(layers.Conv2D(256, (3, 3), strides=1, activation='relu', padding='same'))
model.add(layers.MaxPooling2D((3, 3), strides=2))

model.add(layers.Flatten())
# model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.5))

# model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.5))
# model.add(layers.Dense(1000, activation='softmax'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_30 (Conv2D)          (None, 26, 26, 32)        896       
                                                                 
 lambda_16 (Lambda)          (None, 26, 26, 32)        0         
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 25, 25, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_31 (Conv2D)          (None, 25, 25, 64)        18496     
                                                                 
 lambda_17 (Lambda)          (None, 25, 25, 64)        0         
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 24, 24, 64)       0         
 g2D)                                                

In [26]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, batch_size=128, epochs=10, validation_split=0.2)

Epoch 1/10
375/375 [==============================] - 17s 42ms/step - loss: 1.5387 - accuracy: 0.4440 - val_loss: 0.7574 - val_accuracy: 0.7570
Epoch 2/10
375/375 [==============================] - 15s 41ms/step - loss: 1.2025 - accuracy: 0.5437 - val_loss: 0.6534 - val_accuracy: 0.7712
Epoch 3/10
375/375 [==============================] - 15s 41ms/step - loss: 1.0854 - accuracy: 0.5839 - val_loss: 0.6104 - val_accuracy: 0.7740
Epoch 4/10
375/375 [==============================] - 15s 41ms/step - loss: 1.0406 - accuracy: 0.5984 - val_loss: 0.5770 - val_accuracy: 0.7782
Epoch 5/10
375/375 [==============================] - 15s 41ms/step - loss: 1.0025 - accuracy: 0.6128 - val_loss: 0.5961 - val_accuracy: 0.7641
Epoch 6/10
375/375 [==============================] - 15s 41ms/step - loss: 0.9814 - accuracy: 0.6258 - val_loss: 0.5679 - val_accuracy: 0.7785
Epoch 7/10
375/375 [==============================] - 15s 41ms/step - loss: 0.9555 - accuracy: 0.6344 - val_loss: 0.5432 - val_accuracy: